In [58]:
import torch
import h5py
import os
import numpy as np
from pathlib import Path
import pandas as pd
from dataclasses import dataclass, field
import torch_geometric
from torch_geometric.data import Data, Dataset
from dataclasses import dataclass
import utils
from imblearn.over_sampling import SMOTE
from torch_geometric.utils import from_networkx
from scipy.signal import resample
import networkx as nx
from joblib import Parallel, delayed
import yaml
import fnmatch
from datetime import datetime

In [93]:
@dataclass
class HDFDataset_Writer:
    seizure_lookback: int = 600
    sample_timestep: int = 5
    inter_overlap: int = 0
    preictal_overlap: int = 0
    ictal_overlap: int = 0
    downsample: int = None
    sampling_f: int = 256
    self_loops: bool = False
    balance: bool = False
    smote: bool = False
    buffer_time: int = 15
    connectivity_metric : str = "plv"
    used_classes_dict: dict[str] = field(
        default_factory=lambda: {"interictal": True, "preictal": True, "ictal": True}
    )
    npy_dataset_path: str = "npy_dataset"
    event_tables_path: str = "event_tables"
    cache_folder: str = "cache"
    
    
    def _find_matching_configs(self, current_config):
        
        def find_yaml_files(directory):
            yaml_files = []
            for root, dirs, files in os.walk(directory):
                for file in files:
                    if fnmatch.fnmatch(file, "*.yaml"):
                        yaml_files.append(os.path.join(root, file))
            return yaml_files
        
        config_files = find_yaml_files(self.cache_folder)
        for config_file in config_files:
            with open(config_file) as f:
                config_dict = yaml.load(f, Loader=yaml.FullLoader)
                if config_dict == current_config:
                    print(f"Found matching config file {config_file}")
                    return True
        return False
        
    
    def _create_config_dict(self):
        dataclass_keys = list(self.__dataclass_fields__.keys())
        dict_values = [self.__getattribute__(key) for key in dataclass_keys]
        initial_config_dict = dict(zip(dataclass_keys, dict_values))
        return initial_config_dict
    
    def _create_config_file(self,config_dict, dataset_folder_path):
        with open(os.path.join(dataset_folder_path, "config.yaml"), "w") as f:
            yaml.dump(config_dict, f)
    
    def _get_event_tables(self, patient_name: str) -> tuple[dict, dict]:
        """Read events for given patient into start and stop times lists from .csv extracted files.
        Args:
            patient_name: (str) Name of the patient to get events for.
        Returns:
            start_events_dict: (dict) Dictionary with start events for given patient.
            stop_events_dict: (dict) Dictionary with stop events for given patient.
        """

        event_table_list = os.listdir(self.event_tables_path)
        patient_event_tables = [
            os.path.join(self.event_tables_path, ev_table)
            for ev_table in event_table_list
            if patient_name in ev_table
        ]
        patient_event_tables = sorted(patient_event_tables)
        patient_start_table = patient_event_tables[
            0
        ]  ## done terribly, but it has to be so for win/linux compat
        patient_stop_table = patient_event_tables[1]
        start_events_dict = pd.read_csv(patient_start_table).to_dict("index")
        stop_events_dict = pd.read_csv(patient_stop_table).to_dict("index")
        return start_events_dict, stop_events_dict

    def _get_recording_events(self, events_dict, recording) -> list[int]:
        """Read seizure times into list from event_dict.
        Args:
            events_dict: (dict) Dictionary with events for given patient.
            recording: (str) Name of the recording to get events for.
        Returns:
            recording_events: (list) List of seizure event start and stop time for given recording.
        """
        recording_list = list(events_dict[recording + ".edf"].values())
        recording_events = [int(x) for x in recording_list if not np.isnan(x)]
        return recording_events

    def _create_edge_idx_and_attributes(
        self, connectivity_matrix: np.ndarray, threshold: int = 0.0
    ) -> tuple[np.ndarray, np.ndarray]:
        """Create adjacency matrix from connectivity matrix. Edges are created for values above threshold.
        If the edge is created, it has an attribute "weight" with the value of the connectivity measure associated.
        Args:
            connectivity_matrix: (np.ndarray) Array with connectivity values.
            threshold: (float) Threshold for creating edges. (default: 0.0)
        Returns:
            edge_index: (np.ndarray) Array with edge indices.
            edge_weights: (np.ndarray) Array with edge weights.
        """
        result_graph = nx.graph.Graph()
        n_nodes = connectivity_matrix.shape[0]
        result_graph.add_nodes_from(range(n_nodes))
        edge_tuples = [
            (i, j)
            for i in range(n_nodes)
            for j in range(n_nodes)
            if connectivity_matrix[i, j] > threshold
        ]
        result_graph.add_edges_from(edge_tuples)
        edge_index = nx.convert_matrix.to_numpy_array(result_graph)
        # connection_indices = np.where(edge_index==1)
        # edge_weights = connectivity_matrix[connection_indices] ## ??

        return edge_index

    def _features_to_data_list(self, features, edges, labels, edge_weights=None):
        """Converts features, edges and labels to list of torch_geometric.data.Data objects.
        Args:
            features: (np.ndarray) Array with features.
            edges: (np.ndarray) Array with edges.
            labels: (np.ndarray) Array with labels.
        Returns:
            data_list: (list) List of torch_geometric.data.Data objects.
        """
        data_list = [
            Data(
                x=features[i],
                edge_index=edges[i],
                edge_attr=edge_weights[i],
                y=labels[i],
                # time=time_label[i],
            )
            for i in range(len(features))
        ]
        return data_list

    def _apply_smote(self, features, labels):
        """Performs SMOTE oversampling on the dataset. Implemented for preictal vs ictal scenarion only.
        Args:
            features: (np.ndarray) Array with features.
            labels: (np.ndarray) Array with labels.
        Returns:
            x_train_smote: (np.ndarray) Array with SMOTE oversampled features.
            y_train_smote: (np.ndarray) Array with SMOTE oversampled labels.
        """
        dim_1, dim_2, dim_3 = features.shape

        new_dim = dim_1 * dim_2
        new_x_train = features.reshape(new_dim, dim_3)
        new_y_train = []
        for i in range(len(labels)):
            new_y_train.extend([labels[i]] * dim_2)

        new_y_train = np.array(new_y_train)

        # transform the dataset
        oversample = SMOTE(random_state=42)
        x_train, y_train = oversample.fit_resample(new_x_train, new_y_train)
        x_train_smote = x_train.reshape(int(x_train.shape[0] / dim_2), dim_2, dim_3)
        y_train_smote = []
        for i in range(int(x_train.shape[0] / dim_2)):
            # print(i)
            value_list = list(y_train.reshape(int(x_train.shape[0] / dim_2), dim_2)[i])
            # print(list(set(value_list)))
            y_train_smote.extend(list(set(value_list)))
            ## Check: if there is any different value in a list
            if len(set(value_list)) != 1:
                print(
                    "\n\n********* STOP: THERE IS SOMETHING WRONG IN TRAIN ******\n\n"
                )
        y_train_smote = np.array(y_train_smote)
        # print(np.unique(y_train_smote,return_counts=True))
        return x_train_smote, y_train_smote

    def _get_labels_features_edge_weights_seizure(self, patient):
        """Method to extract features, labels and edge weights for seizure and interictal samples."""

        event_tables = self._get_event_tables(
            patient
        )  # extract start and stop of seizure for patient
        patient_path = os.path.join(self.npy_dataset_path, patient)
        recording_list = [
            recording
            for recording in os.listdir(patient_path)
            if "seizures" in recording
        ]
        self.hdf5_file.create_group(patient)
        for n, record in enumerate(
            recording_list
        ):  # iterate over recordings for a patient
            recording_path = os.path.join(patient_path, record)
            record = record.replace(
                "seizures_", ""
            )  ## some magic to get it properly working with event tables
            record_id = record.split(".npy")[0]  #  get record id
            start_event_tables = self._get_recording_events(
                event_tables[0], record_id
            )  # get start events
            stop_event_tables = self._get_recording_events(
                event_tables[1], record_id
            )  # get stop events
            data_array = np.load(recording_path)  # load the recording

            features, labels, time_labels = utils.extract_training_data_and_labels(
                data_array,
                start_event_tables,
                stop_event_tables,
                fs=self.sampling_f,
                seizure_lookback=self.seizure_lookback,
                sample_timestep=self.sample_timestep,
                preictal_overlap=self.preictal_overlap,
                ictal_overlap=self.ictal_overlap,
                buffer_time=self.buffer_time,
            )

            if features is None:
                print(
                    f"Skipping the recording {record} patients {patient} cuz features are none"
                )
                continue

            features = features.squeeze(2)
            sampling_f = self.sampling_f if self.downsample is None else self.downsample
            if self.downsample:
                new_sample_count = int(self.downsample * self.sample_timestep)
                features = resample(features, new_sample_count, axis=2)
                
            conn_matrix_list = [
                self.connectivity_function(feature, sampling_f) for feature in features
            ]
            edge_idx = np.stack(
                [
                    self._create_edge_idx_and_attributes(
                        conn_matrix, threshold=np.mean(conn_matrix)
                    )
                    for conn_matrix in conn_matrix_list
                ]
            )
            edge_idx = np.stack(edge_idx)
            
            if self.smote:
                features, labels = self._apply_smote(features, labels)
            labels = labels.reshape((labels.shape[0], 1)).astype(np.float32)

            features_patient = (
                features if n == 0 else np.concatenate([features_patient, features])
            )
            labels_patient = (
                labels if n == 0 else np.concatenate([labels_patient, labels])
            )
            edge_idx_patient = (
                edge_idx if n == 0 else np.concatenate([edge_idx_patient, edge_idx])
            )
        #     edge_weights_patient = edge_weights if n == 0 else np.concatenate([edge_weights_patient, edge_weights])

        self.hdf5_file[patient].create_dataset("features", data=features_patient)
        self.hdf5_file[patient].create_dataset("labels", data=labels_patient)
        self.hdf5_file[patient].create_dataset("edge_idx", data=edge_idx_patient)

    def get_dataset(self):
        
        
        folder_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        dataset_folder = os.path.join(self.cache_folder, folder_name)
        dataset_path = os.path.join(dataset_folder, "dataset.hdf5")
        current_config = self._create_config_dict()    
        if self._find_matching_configs(current_config):
            print(
                f"Dataset already exists. Dataset not created."
            )
            return None
        
        os.makedirs(dataset_folder, exist_ok=True)
        self._create_config_file(current_config, dataset_folder)
        self.hdf5_file = h5py.File(dataset_path, "w")
        patient_list = os.listdir(self.npy_dataset_path)
        self.connectivity_function = utils.compute_plv_matrix if self.connectivity_metric == "plv" else utils.compute_spect_corr_matrix
        try:
            if self.smote:
                for patient in patient_list:
                    self._get_labels_features_edge_weights_seizure(patient)
            else:
                Parallel(n_jobs=6, require="sharedmem")(
                    delayed(self._get_labels_features_edge_weights_seizure)(patient)
                    for patient in patient_list
                )

            self.hdf5_file.close()
            print(f"Dataset created in folder {dataset_folder}.")
        except:
            self.hdf5_file.close()
            os.remove(dataset_path)
            os.remove(os.path.join(dataset_folder, "config.yaml"))
            os.rmdir(dataset_folder)
            raise Exception("Dataset creation failed. Dataset deleted.")

In [94]:
torch_geometric.seed_everything(42)
hdf_writer = HDFDataset_Writer(
    npy_dataset_path="data/npy_data_full",
    event_tables_path="data/event_tables",
    cache_folder="cache",
    seizure_lookback=600,
    sample_timestep=9,
    downsample=60,
    connectivity_metric="corr"
)

In [96]:
torch_geometric.seed_everything(42)
hdf_writer.get_dataset()

Found matching config file cache/2023-07-03_10-32-22/config.yaml
Dataset already exists. Dataset not created.


In [32]:
import yaml
with open("config.yaml", "w") as f:
    ## dump class attributes to yaml
    yaml.dump(hdf_writer.__dict__, f)

In [96]:
import hashlib
def hashfile(file):
  
    # A arbitrary (but fixed) buffer
    # size (change accordingly)
    # 65536 = 65536 bytes = 64 kilobytes
    BUF_SIZE = 65536
  
    # Initializing the sha256() method
    sha256 = hashlib.sha256()
  
    # Opening the file provided as
    # the first commandline argument
    with open(file, 'rb') as f:
         
        while True:
             
            # reading data = BUF_SIZE from
            # the file and saving it in a
            # variable
            data = f.read(BUF_SIZE)
  
            # True if eof = 1
            if not data:
                break
      
            # Passing that data to that sh256 hash
            # function (updating the function with
            # that data)
            sha256.update(data)
  
      
    # sha256.hexdigest() hashes all the input
    # data passed to the sha256() via sha256.update()
    # Acts as a finalize method, after which
    # all the input data gets hashed hexdigest()
    # hashes the data, and returns the output
    # in hexadecimal format
    return sha256.hexdigest()
f1_hash = hashfile("cache/lookback_600_timestep_9_overlap_0_0_0_downsample_60_smote_False_self_loops_False_balance_False/dataset.hdf5")
f2_hash = hashfile("cache1/lookback_600_timestep_9_overlap_0_0_0_downsample_60_smote_False_self_loops_False_balance_False/dataset.hdf5")
  
# Doing primitive string comparison to
# check whether the two hashes match or not
if f1_hash == f2_hash:
    print("Both files are same")
    print(f"Hash: {f1_hash}")
 
else:
    print("Files are different!")
    print(f"Hash of File 1: {f1_hash}")
    print(f"Hash of File 2: {f2_hash}")

Both files are same
Hash: ea97fb188f2d96fc40a116e195cbd1147fe559e6a494f34af4ad20cc700df583


In [83]:
created_hdf = h5py.File("cache1/lookback_600_timestep_9_overlap_0_0_0_downsample_60_smote_False_self_loops_False_balance_False/dataset.hdf5", "r")

In [90]:
np.unique(created_hdf["chb02"]["labels"],return_counts=True)

(array([0., 1.], dtype=float32), array([143,  19]))

In [ ]:
example_graph = nx.Graph()
example_graph.add_nodes_from(range(10))
example_graph.add_edges_from([(0,1),(1,2),(2,3),(3,4),(4,5),(5,6),(6,7),(7,8),(8,9)])
nx.draw(example_graph,with_labels=True)
numpy_adj = nx.convert_matrix.to_numpy_array(example_graph)
torch_geometric.utils.dense_to_sparse(torch.from_numpy(numpy_adj))[0]

In [ ]:
dummy_hdf.close()

In [ ]:
dummy_hdf = h5py.File("dummy.hdf5", "w")

In [ ]:
dummy_hdf.create_group("patient_1")

In [ ]:
features = np.random.rand(10, 10, 10)
labels = np.random.randint(0, 2, (10, 1))
edge_idx = np.random.randint(0, 10, (10, 2))
edge_weights = np.random.rand(10, 1)

In [ ]:
dummy_hdf['patient_1'].create_dataset("features", data=features, maxshape=(None, 10, 10))
dummy_hdf['patient_1'].create_dataset("labels", data=labels, maxshape=(None, 1))
dummy_hdf['patient_1'].create_dataset("edge_idx", data=edge_idx, maxshape=(None, 2))
dummy_hdf['patient_1'].create_dataset("edge_weights", data=edge_weights, maxshape=(None, 1))

In [ ]:
data_obj = Data(x=torch.tensor(features[0]), edge_index=torch.tensor(edge_idx[0]).T, edge_attr=torch.tensor(edge_weights[0]), y=torch.tensor(labels[0]))

In [ ]:
# append to dataset
# dummy_hdf['patient_1']['features'].resize((dummy_hdf['patient_1']['features'].shape[0] + features.shape[0]), axis=0)z
# dummy_hdf['patient_1']['features'][-features.shape[0]:] = features
dummy_hdf['patient_1']['labels'].resize((dummy_hdf['patient_1']['labels'].shape[0] + labels.shape[0]), axis=0)
dummy_hdf['patient_1']['labels'][-labels.shape[0]:] = labels
dummy_hdf['patient_1']['edge_idx'].resize((dummy_hdf['patient_1']['edge_idx'].shape[0] + edge_idx.shape[0]), axis=0)
dummy_hdf['patient_1']['edge_idx'][-edge_idx.shape[0]:] = edge_idx

In [ ]:
dummy_hdf.close()

In [ ]:
read_hdf = h5py.File("dummy.hdf5", "r")

In [ ]:
grp = dummy_hdf["patient_1"]

In [ ]:
np.array_equal(grp['labels'][:],np.concatenate([labels,labels],axis=0))